In [1]:
import pandas as pd
import requests

In [2]:
query_user = "What's the cheapeast journey from London to Paris and back departing september 10th 2023 for one adult and returning on the 12th september 2023"

query = f'''Based on the user query: {query_user} do the following: 
Return the relevant information to search for the flights online.
Some journeys are indirect meaning they have multiple legs, remember to factor this into your calculation. The prices quoted are the total price for the journey.
'''


In [13]:
# read keys
import yaml
import os

def read_config():
    # Get the directory of the current script
    # script_dir = os.path.dirname(os.path.realpath(__file__))
    script_dir = "../src/"

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        try:
            configs = yaml.safe_load(stream)
            api_key = configs['amadeues_flights']['api_key']
            api_secret = configs['amadeues_flights']['api_secret']
            openai_key = configs['openai']['openai_key']
            hugging_api_key = configs['huggingfacehub']['hugging_api_key']
            return api_key, api_secret, openai_key, hugging_api_key
        except yaml.YAMLError as exc:
            print(exc)
            
    return api_key, api_secret, openai_key

class SingletonToken:
    __token = None

    @classmethod
    def set_token(cls, token):
        cls.__token = token

    @classmethod
    def get_token(cls):
        return cls.__token

api_key, api_secret, openai_key, hugging_api_key = read_config()

In [4]:
def journey_data(response_flights_data, response_airline_lookup_data):
    # Load the data into a DataFrame
    df = pd.DataFrame(response_flights_data)
    df_airline_codes = pd.json_normalize(response_airline_lookup_data)
    
    # Extract itineraries, validatingAirlineCodes, price (total and currency) and id into separate dataframes
    df_itineraries = df[['id', 'itineraries']].explode('itineraries').reset_index(drop=True)
    
    # In the itineraries column, each cell is a dictionary. So, we need to convert those dictionaries into separate columns.
    df_itineraries = df_itineraries.join(pd.json_normalize(df_itineraries['itineraries'])).drop(columns='itineraries')
    
    # At this point, 'segments' column is a list of dictionaries where each dictionary represents a leg of the journey.
    # We want each leg to be a separate row in the dataframe. So, explode the 'segments' column.
    df_itineraries = df_itineraries.explode('segments').reset_index(drop=True)
    
    # Add a 'leg_id' column to identify each leg of the journey
    df_itineraries['leg_id'] = df_itineraries.groupby('id').cumcount() + 1
    
    # Now, convert the dictionaries in the 'segments' column into separate columns
    df_segments = pd.json_normalize(df_itineraries['segments'])
    
    # To avoid overlapping columns, add a prefix to the column names of the new dataframe
    df_segments.columns = ['flight_' + str(col) for col in df_segments.columns]
    
    # Now join the original dataframe with the new one
    df_itineraries = df_itineraries.join(df_segments).drop(columns='segments')
    
    df_validatingAirlineCodes = df[['id', 'validatingAirlineCodes']]
    
    # For the price column, we only need total and currency. So, extract only those into a new dataframe
    df_price = df['price'].apply(pd.Series)[['total', 'currency']]
    df_price['id'] = df['id']
    
    # Now join these dataframes on the 'id' column
    df_flights = pd.merge(df_itineraries, df_validatingAirlineCodes, on='id')
    df_flights = pd.merge(df_flights, df_price, on='id')
    
    # Create a new column for the total number of legs per journey
    df_flights['total_legs'] = df_flights.groupby('id')['leg_id'].transform('max')
    
    df_flights = df_flights.merge(right=df_airline_codes, how='left', left_on="flight_operating.carrierCode", right_on="iataCode")
    df_flights.rename(columns={"id":"journey_id", "commonName":"airline" }, inplace=True)

    df_flights.drop(columns=["flight_id", "validatingAirlineCodes", "businessName", "flight_operating.carrierCode", "flight_aircraft.code"], inplace=True)

    df_flights.columns = df_flights.columns.str.replace('.', '_')

    return df_flights

In [5]:
from sqlalchemy import create_engine
import pandas as pd
from langchain import SQLDatabase

def load_data(df_flights):
    engine = create_engine('sqlite:///:memory:')

    # Write the data to the SQLite database
    df_flights.to_sql('flights', engine, if_exists='replace', index=False)
    # Check if the data was loaded correctly
    df_loaded = pd.read_sql('SELECT * FROM flights', engine)
    db = SQLDatabase(engine)
    return db

In [6]:
import json
import openai

def get_args(query_user: str) -> str:
    # OpenAI function calling

    """Get's arguments based on client query, returns num_adults, departureDate, destinationLocationCode, originLocationCode"""
    
    function_call = [
    {
      "name": "search_for_flights",
      "description": "Requests flight data from Amadeus API and writes to SQLite database",
      "parameters": {
        "type": "object",
        "properties": {
            "num_adults":{
                "type":"integer",
                "description": '''Based on the query, respond with the number of adults'''
            },
            "departureDate": {
                "type":"string",
                "description": '''Based on the query, respond with the Departure Date. Dates are specified in the ISO 8601 YYYY-MM-DD format. '''
            },
            "returnDate": {
                "type":"string",
                "description": '''Based on the query, respond with the Return Date. Dates are specified in the ISO 8601 YYYY-MM-DD format. '''
            },
            "destinationLocationCode":{
                "type":"string",
                "description": '''Based on the query, respond with an airport IATA code from the city which the traveler is going. E.g CDG for Charles de Gaulle Airport'''
            },
          "originLocationCode": {
            "type": "string",
            "description": '''Based on the query, respond with an airport IATA code from the city which the traveler will depart from. E.g CDG for Charles de Gaulle Airport'''
          },

        },
        "required": ["destinationLocationCode", "originLocationCode", "departureDate", "returnDate", "num_adults"]
      }
    }
    ]
    
    openai.api_key = openai_key

    message = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": query_user}],
        functions = function_call,
        function_call = 'auto',
        temperature=0
    )
    response_message = message["choices"][0]["message"]["function_call"]["arguments"]

    parsed_data = json.loads(response_message)

    # Accessing variables
    num_adults = parsed_data['num_adults']
    departureDate = parsed_data['departureDate']
    returnDate = parsed_data['returnDate']
    destinationLocationCode = parsed_data['destinationLocationCode']
    originLocationCode = parsed_data['originLocationCode']
    
    print("Number of Adults: ", num_adults)
    print("Departure Date: ", departureDate)
    print("Return Date: ", returnDate)
    print("Destination Location Code: ", destinationLocationCode)
    print("Origin Location Code: ", originLocationCode)

    return num_adults, departureDate, returnDate, destinationLocationCode, originLocationCode

num_adults, departureDate, returnDate, destinationLocationCode, originLocationCode = get_args(query_user)

Number of Adults:  1
Departure Date:  2023-09-10
Return Date:  2023-09-12
Destination Location Code:  CDG
Origin Location Code:  LHR


In [7]:
from amadeus import Client, ResponseError
from datetime import datetime
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain.tools import tool
from langchain import LLMMathChain
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain


def search_for_flights(originLocationCode, destinationLocationCode, departureDate, returnDate, num_adults) -> SQLDatabase:
    """Requets flight data from Amadeus API and writes to sqllite database and run SQLDatabaseQuery
    originLocationCode: Based on the query, respond with the iataCode for the origin airport,
    destinationLocationCode: Based on the query, respond with the iataCode for the destination airport,
    departureDate: Based on the query, respond with the departure date,
    num_adults: Based on the query, respond with the number of adults
    """

    # Assuming you've defined api_key and api_secret somewhere else
    amadeus = Client(client_id=api_key, client_secret=api_secret)

    # Defining the parameters for the flight
    params = {
        'originLocationCode': originLocationCode,
        'destinationLocationCode': destinationLocationCode,
        'departureDate': departureDate,
        'returnDate': returnDate,
        'adults': num_adults
        # add or modify other parameters according to your needs
    }
    
    try:
        response_flights = amadeus.shopping.flight_offers_search.get(**params)
        
    except ResponseError as error:
        print(f"ResponseError occurred flights: {error}")
        print(f"Error code flights: {error.code}")
        print(f"Error message flights: {error.description}")
        return []  # return an empty list in case of an error

    try:
        response_airline_lookup = amadeus.reference_data.airlines.get()

    except ResponseError as error:
        print(f"ResponseError occurred airline lookup: {error}")
        print(f"Error code airline lookup: {error.code}")
        print(f"Error message airline lookup: {error.description}")

    df_flights = journey_data(response_flights.data, response_airline_lookup.data)
    db = load_data(df_flights)

    return db, df_flights

db, df_flights = search_for_flights(originLocationCode, destinationLocationCode, departureDate, returnDate, num_adults)
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653 entries, 0 to 652
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   journey_id                 653 non-null    object
 1   duration                   653 non-null    object
 2   leg_id                     653 non-null    int64 
 3   flight_carrierCode         653 non-null    object
 4   flight_number              653 non-null    object
 5   flight_duration            653 non-null    object
 6   flight_numberOfStops       653 non-null    int64 
 7   flight_blacklistedInEU     653 non-null    bool  
 8   flight_departure_iataCode  653 non-null    object
 9   flight_departure_terminal  441 non-null    object
 10  flight_departure_at        653 non-null    object
 11  flight_arrival_iataCode    653 non-null    object
 12  flight_arrival_terminal    441 non-null    object
 13  flight_arrival_at          653 non-null    object
 14  total     

In [8]:
# # Assuming you've defined api_key and api_secret somewhere else
# amadeus = Client(client_id=api_key, client_secret=api_secret)

# # Defining the parameters for the flight
# params = {
#     'originLocationCode': 'LHR',
#     'destinationLocationCode': 'CDG',
#     'departureDate': '2023-09-10',  # API might require specific date format
#     'returnDate': '2023-12-10',
#     'adults': 1,
#     'children': 1,
#     'infants': 1    
#     # add or modify other parameters according to your needs
# }

# try:
#     response_flights = amadeus.shopping.flight_offers_search.get(**params)
    
# except ResponseError as error:
#     print(f"ResponseError occurred flights: {error}")
#     print(f"Error code flights: {error.code}")
#     print(f"Error message flights: {error.description}")
#     # return []  # return an empty list in case of an error

# try:
#     response_airline_lookup = amadeus.reference_data.airlines.get()

# except ResponseError as error:
#     print(f"ResponseError occurred airline lookup: {error}")
#     print(f"Error code airline lookup: {error.code}")
#     print(f"Error message airline lookup: {error.description}")

# df_flights = journey_data(response_flights.data, response_airline_lookup.data)
# df_flights
# # db = load_data(df_flights)

In [9]:
# df_flights.to_csv("../src/flights.csv")

In [11]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0, model="gpt-4-0613", openai_api_key=openai_key)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

agent_executor.run(query)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: flights
Thought:The 'flights' table seems to be the most relevant one for this query. I should check its schema to understand the structure and the data it contains.
Action: sql_db_schema
Action Input: "flights"
Observation: 
CREATE TABLE flights (
	journey_id TEXT, 
	duration TEXT, 
	leg_id BIGINT, 
	"flight_carrierCode" TEXT, 
	flight_number TEXT, 
	flight_duration TEXT, 
	"flight_numberOfStops" BIGINT, 
	"flight_blacklistedInEU" BOOLEAN, 
	"flight_departure_iataCode" TEXT, 
	flight_departure_terminal TEXT, 
	flight_departure_at TEXT, 
	"flight_arrival_iataCode" TEXT, 
	flight_arrival_terminal TEXT, 
	flight_arrival_at TEXT, 
	total TEXT, 
	currency TEXT, 
	total_legs BIGINT, 
	type TEXT, 
	"iataCode" TEXT, 
	"icaoCode" TEXT, 
	airline TEXT
)

/*
3 rows from flights table:
journey_id	duration	leg_id	flight_carrierCode	flight_number	flight_duration	flight_numberOfStops	flight_blacklistedIn

'Here are the details of the 10 cheapest journeys from London to Paris and back:\n\n1. Journey ID: 3, Airline: AF, Flight Number: 1381, Departure: LHR at 2023-09-10T06:20:00, Arrival: CDG at 2023-09-10T08:35:00, Return Flight Number: 1380, Return Departure: CDG at 2023-09-12T20:55:00, Return Arrival: LHR at 2023-09-12T21:20:00\n2. Journey ID: 6, Airline: AF, Flight Number: 1381, Departure: LHR at 2023-09-10T06:20:00, Arrival: CDG at 2023-09-10T08:35:00, Return Flight Number: 1780, Return Departure: CDG at 2023-09-12T13:20:00, Return Arrival: LHR at 2023-09-12T13:45:00\n3. Journey ID: 7, Airline: AF, Flight Number: 1381, Departure: LHR at 2023-09-10T06:20:00, Arrival: CDG at 2023-09-10T08:35:00, Return Flight Number: 1280, Return Departure: CDG at 2023-09-12T16:10:00, Return Arrival: LHR at 2023-09-12T16:35:00\n4. Journey ID: 8, Airline: AF, Flight Number: 1381, Departure: LHR at 2023-09-10T06:20:00, Arrival: CDG at 2023-09-10T08:35:00, Return Flight Number: 1580, Return Departure: CDG 

In [46]:
import os 
from langchain import HuggingFaceHub

repo_id = "baichuan-inc/Baichuan-13B-Chat"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hugging_api_key

In [47]:
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5})

In [48]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)
db_chain.run(query)



> Entering new SQLDatabaseChain chain...
Based on the user query: What's the cheapeast journey from London to Paris and back departing september 10th 2023 for one adult and returning on the 12th september 2023 do the following: 
Return the relevant information to search for the flights online.
Some journeys are indirect meaning they have multiple legs, remember to factor this into your calculation. The prices quoted are the total price for the journey.

SQLQuery:

ValueError: Error raised by inference API: Model baichuan-inc/Baichuan-13B-Chat time out